In [17]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

In [18]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.head()
ttrain=train.drop('Attrition',axis=1)
all_data = pd.concat((ttrain.loc[:,'Age':],
                      test.loc[:,'Age':]))
all_data = pd.get_dummies(all_data)
all_data.drop('EmployeeNumber',axis=1,inplace=True)
#all_data.drop(['Behaviour','EmployeeNumber','OverTime_Yes','MaritalStatus_Single','JobRole_Sales Representative','Gender_Male','BusinessTravel_Travel_Rarely','EducationField_Medical','Department_Human Resources'],axis=1,inplace=True)


In [19]:
#featureSelection

In [20]:
from sklearn.preprocessing import KBinsDiscretizer
est = KBinsDiscretizer(n_bins=6, encode='ordinal', strategy='uniform')
X=np.array(all_data['Age'])
X=X.reshape(-1,1)
est.fit(X)
all_data['Age'] = est.transform(X)

In [21]:
est = KBinsDiscretizer(n_bins=19, encode='ordinal', strategy='uniform')
X=np.array(all_data['MonthlyIncome'])
X=X.reshape(-1,1)
est.fit(X)
all_data['MonthlyIncome'] = est.transform(X)

In [22]:
all_data['Satisfaction']=(all_data['EnvironmentSatisfaction']+all_data['JobSatisfaction'])/2

In [23]:
all_data.drop(['EnvironmentSatisfaction','JobSatisfaction'],axis=1,inplace=True)

In [24]:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.Attrition

In [26]:
# Some useful parameters which will come in handy later on
ntrain = X_train.shape[0]
ntest = X_test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = StratifiedKFold(ntrain, n_splits = 5, random_state=SEED)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
# Class to extend XGboost classifer

TypeError: __init__() got multiple values for argument 'n_splits'